# Prepare Dataset

In [ ]:
import re
import csv
import glob
import requests
import unicodedata
from tqdm import tqdm
from threading import Thread
from os import makedirs, path

ROOT_PATH = path.abspath(path.dirname(path.dirname(__file__)))

DATA_PATH = path.join(ROOT_PATH, 'data')
RAW_DATA_PATH = path.join(DATA_PATH, 'raw')
FINAL_DATA_PATH = path.join(DATA_PATH, 'final')
ALL_DATA_FILE_PATH = path.join(FINAL_DATA_PATH, 'all_data.csv')
ASSISTANCE_FILE_PATH = path.join(FINAL_DATA_PATH, 'Cidadania e assistência social.csv')
RAW_ADDRESSES_FILE_PATH = path.join(FINAL_DATA_PATH, 'enderecos.csv')
ADDRESSES_FILE_PATH = path.join(FINAL_DATA_PATH, 'enderecos_sao_paulo.csv')

CEP_COLUMN = "cep"
TOPIC_COLUMN = "assunto"

COLUMNS_TO_KEEP = [
"data de abertura",
"canal",
"orgao",
"tema",
"assunto",
"servico",
"bairro",
"cep",
"distrito",
"latitude",
"logradouro",
"longitude",
"quadra",
"numero",
]

In [ ]:
makedirs(DATA_PATH, exist_ok=True)
makedirs(RAW_DATA_PATH, exist_ok=True)
makedirs(FINAL_DATA_PATH, exist_ok=True)

In [ ]:
response = requests.get("http://dados.prefeitura.sp.gov.br/pt_PT/dataset/dados-do-sp156")

urls = re.findall(r'href="(.*\.csv)"', response.text)

In [ ]:
progress_bar = tqdm(desc="Downloading SP156 Relatories", total=len(urls))

def download_csv(file_name: str, url: str):
    response = requests.get(url)

    file_path = path.join(RAW_DATA_PATH, f'{file_name}.csv')

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(response.text)

    if progress_bar is not None:
        progress_bar.update()


In [ ]:
threads = []

for index, url in enumerate(urls):
    file_name = f'file_{index+1}'
    thread = Thread(target=download_csv, args=(file_name, url))
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()

In [ ]:
path_name = path.join(RAW_DATA_PATH, '*.csv')

files = glob.glob(path_name)

downloaded_csvs = sorted(files, key=lambda file: int(re.findall('(\d+)', file)[0]))

valid_csvs = downloaded_csvs[:-6] # remove csvs from 2014 to 2012


In [ ]:
def normalize_text(text:str):
   return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn').lower().strip()

def open_csv_writer(filepath:str):
    file_descriptor = open(filepath, '+w', encoding="utf8")

    return csv.writer(file_descriptor, delimiter=';')

def open_csv_reader(filepath:str):
    file_descriptor = open(filepath, 'r', encoding="utf8")

    return csv.reader(file_descriptor, delimiter=';')

In [ ]:
all_data_csv_writer = open_csv_writer(ALL_DATA_FILE_PATH)

all_data_csv_writer.writerow(COLUMNS_TO_KEEP)

for csv_file in tqdm(valid_csvs[:-6], desc="Concatenating rows to a single file"):
    csv_reader = open_csv_reader(csv_file)

    headers = next(csv_reader)

    normalized_headers = list(map(lambda head: normalize_text(head), headers))

    indexes = list(map(lambda column: None if column not in normalized_headers else normalized_headers.index(column), COLUMNS_TO_KEEP))

    for row in csv_reader:
        mapped_row = map(lambda index:None if index is None else row[index], indexes)

        all_data_csv_writer.writerow(list(mapped_row))

In [ ]:
all_data_csv = open_csv_reader(ALL_DATA_FILE_PATH)

header = next(all_data_csv)

theme_index = header.index('tema')
file_map = dict()

for row in tqdm(all_data_csv, desc="Separating rows to multiple files"):
    theme = row[theme_index].replace('/', '').replace('\\', '')

    if not file_map.get(theme):
        theme_file = path.join(FINAL_DATA_PATH, f'{theme}.csv')

        file_map[theme] = open_csv_writer(theme_file)
        file_map[theme].writerow(header)

    file_map[theme].writerow(row)


In [ ]:
def str_to_none_if_empty(string:str):
    string = string.strip()

    if len(string) == 0:
        return None
    
    return string

In [ ]:
reader = open_csv_reader(ASSISTANCE_FILE_PATH)

header = next(reader)

CEP_INDEX = header.index(CEP_COLUMN)
TOPIC_INDEX = header.index(TOPIC_COLUMN)

cep_set = set()

for row in reader:
    if (row[TOPIC_INDEX] != 'População ou pessoa em situação de rua'):
        continue

    cep = str_to_none_if_empty(row[CEP_INDEX])

    if cep is not None:
        cep_set.add(cep)

In [ ]:
def fetch_cep_data(cep):
    response = requests.get(f"https://opencep.com/v1/{cep}.json")
    
    if response.status_code != 200:
        return None

    data = response.json()
    
    try:
        return (data['cep'], data['logradouro'], data['bairro'], data['localidade'])
    except KeyError:
        return None   

In [ ]:
ceps = list(cep_set)
num_ceps_per_thread = len(ceps) // 8

threads = []

addresses_csv = open_csv_writer(RAW_ADDRESSES_FILE_PATH)

addresses_csv.writerow(('cep', 'logradouro', 'bairro', 'localidade'))

progress_bar = tqdm(total=len(ceps))

def process_ceps(start, end):
    for cep in ceps[start:end]:
        cep_data = fetch_cep_data(cep)

        progress_bar.update()

        if cep_data is None:
            continue
            
        addresses_csv.writerow(cep_data)

In [ ]:
for i in range(8):
    start_index = i * num_ceps_per_thread
    end_index = (i + 1) * num_ceps_per_thread if i != 7 else len(ceps)  

    thread = Thread(target=process_ceps, args=(start_index, end_index))
    
    threads.append(thread)
    
    thread.start()

for thread in threads:
    thread.join()

In [ ]:
lines = open(RAW_ADDRESSES_FILE_PATH, 'r', encoding='utf8', errors='ignore').read().strip().split('\n')

new_lines = []

CEP_REGEX = '\d\d\d\d\d-\d\d\d'

for line in lines:
    if line.startswith('cep'):
        new_lines.append(line)
        continue

    finds = re.findall(CEP_REGEX, line)

    if len(finds) == 1:
        cep = finds[0]

        cep_index = line.index(cep)

        line = line[cep_index + len(cep):]

        line = cep.replace('-', '') + line

        print(line)

    if line.count(',') == 3 and "São Paulo" in line:
        new_lines.append(line.replace('\x00', ''))

open(ADDRESSES_FILE_PATH, 'w+').write('\n'.join(new_lines))